In [2]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K

import matplotlib.pyplot as plt
import os
import sys

import numpy as np


class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
                            optimizer=optimizer,
                            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
                              optimizer=optimizer,
                              metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # ---------------------
                    #  Train Discriminator
                    # ---------------------

                    # Select a random batch of images
                    # idx = np.random.randint(0, X_train.shape[0], batch_size)
                    # imgs = X_train[idx]

                    # Sample noise as generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                    # Generate a batch of new images
                    gen_imgs = self.generator.predict(noise)

                    # Train the critic
                    d_loss_real = self.critic.train_on_batch(imgs, valid)
                    d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                    # Clip critic weights
                    for l in self.critic.layers:
                        weights = l.get_weights()
                        weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                        l.set_weights(weights)

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f] [G loss: %f]" % (epoch, global_step, 1 - d_loss[0], 1 - g_loss[0]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch, global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_wgan_fashion_mnist'):
            os.mkdir('images_wgan_fashion_mnist')
        fig.savefig("images_wgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch, global_step))
        plt.close()

if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=40, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 32)          0         
__________

epoch:0 step:340[D loss: 0.999972] [G loss: 1.000090]
epoch:0 step:345[D loss: 0.999964] [G loss: 1.000106]
epoch:0 step:350[D loss: 0.999981] [G loss: 1.000089]
epoch:0 step:355[D loss: 0.999978] [G loss: 1.000094]
epoch:0 step:360[D loss: 0.999976] [G loss: 1.000094]
epoch:0 step:365[D loss: 0.999976] [G loss: 1.000098]
epoch:0 step:370[D loss: 0.999976] [G loss: 1.000099]
epoch:0 step:375[D loss: 0.999978] [G loss: 1.000108]
epoch:0 step:380[D loss: 0.999969] [G loss: 1.000114]
epoch:0 step:385[D loss: 0.999974] [G loss: 1.000113]
epoch:0 step:390[D loss: 0.999974] [G loss: 1.000112]
epoch:0 step:395[D loss: 0.999977] [G loss: 1.000102]
epoch:0 step:400[D loss: 0.999970] [G loss: 1.000115]
epoch:0 step:405[D loss: 0.999974] [G loss: 1.000111]
epoch:0 step:410[D loss: 0.999962] [G loss: 1.000104]
epoch:0 step:415[D loss: 0.999967] [G loss: 1.000094]
epoch:0 step:420[D loss: 0.999967] [G loss: 1.000101]
epoch:0 step:425[D loss: 0.999969] [G loss: 1.000104]
epoch:0 step:430[D loss: 0.9

epoch:0 step:1100[D loss: 0.999976] [G loss: 1.000070]
epoch:0 step:1105[D loss: 0.999972] [G loss: 1.000071]
epoch:0 step:1110[D loss: 0.999972] [G loss: 1.000070]
epoch:0 step:1115[D loss: 0.999975] [G loss: 1.000069]
epoch:0 step:1120[D loss: 0.999977] [G loss: 1.000074]
epoch:0 step:1125[D loss: 0.999973] [G loss: 1.000069]
epoch:0 step:1130[D loss: 0.999975] [G loss: 1.000073]
epoch:0 step:1135[D loss: 0.999972] [G loss: 1.000074]
epoch:0 step:1140[D loss: 0.999976] [G loss: 1.000069]
epoch:0 step:1145[D loss: 0.999977] [G loss: 1.000062]
epoch:0 step:1150[D loss: 0.999974] [G loss: 1.000075]
epoch:0 step:1155[D loss: 0.999973] [G loss: 1.000073]
epoch:0 step:1160[D loss: 0.999973] [G loss: 1.000067]
epoch:0 step:1165[D loss: 0.999976] [G loss: 1.000074]
epoch:0 step:1170[D loss: 0.999972] [G loss: 1.000070]
epoch:0 step:1175[D loss: 0.999973] [G loss: 1.000063]
epoch:0 step:1180[D loss: 0.999973] [G loss: 1.000070]
epoch:0 step:1185[D loss: 0.999978] [G loss: 1.000064]
epoch:0 st

epoch:0 step:1845[D loss: 0.999974] [G loss: 1.000068]
epoch:0 step:1850[D loss: 0.999978] [G loss: 1.000062]
epoch:0 step:1855[D loss: 0.999978] [G loss: 1.000066]
epoch:0 step:1860[D loss: 0.999979] [G loss: 1.000066]
epoch:0 step:1865[D loss: 0.999975] [G loss: 1.000071]
epoch:0 step:1870[D loss: 0.999973] [G loss: 1.000077]
epoch:0 step:1875[D loss: 0.999968] [G loss: 1.000069]
epoch:0 step:1880[D loss: 0.999972] [G loss: 1.000077]
epoch:0 step:1885[D loss: 0.999973] [G loss: 1.000067]
epoch:0 step:1890[D loss: 0.999979] [G loss: 1.000061]
epoch:0 step:1895[D loss: 0.999973] [G loss: 1.000069]
epoch:0 step:1900[D loss: 0.999969] [G loss: 1.000064]
epoch:0 step:1905[D loss: 0.999973] [G loss: 1.000070]
epoch:0 step:1910[D loss: 0.999968] [G loss: 1.000065]
epoch:0 step:1915[D loss: 0.999973] [G loss: 1.000078]
epoch:0 step:1920[D loss: 0.999971] [G loss: 1.000069]
epoch:0 step:1925[D loss: 0.999979] [G loss: 1.000055]
epoch:0 step:1930[D loss: 0.999972] [G loss: 1.000060]
epoch:0 st

epoch:0 step:2590[D loss: 0.999977] [G loss: 1.000063]
epoch:0 step:2595[D loss: 0.999978] [G loss: 1.000069]
epoch:0 step:2600[D loss: 0.999976] [G loss: 1.000064]
epoch:0 step:2605[D loss: 0.999976] [G loss: 1.000074]
epoch:0 step:2610[D loss: 0.999976] [G loss: 1.000069]
epoch:0 step:2615[D loss: 0.999974] [G loss: 1.000067]
epoch:0 step:2620[D loss: 0.999971] [G loss: 1.000078]
epoch:0 step:2625[D loss: 0.999980] [G loss: 1.000067]
epoch:0 step:2630[D loss: 0.999978] [G loss: 1.000066]
epoch:0 step:2635[D loss: 0.999980] [G loss: 1.000063]
epoch:0 step:2640[D loss: 0.999976] [G loss: 1.000067]
epoch:0 step:2645[D loss: 0.999974] [G loss: 1.000071]
epoch:0 step:2650[D loss: 0.999977] [G loss: 1.000070]
epoch:0 step:2655[D loss: 0.999976] [G loss: 1.000064]
epoch:0 step:2660[D loss: 0.999977] [G loss: 1.000063]
epoch:0 step:2665[D loss: 0.999972] [G loss: 1.000063]
epoch:0 step:2670[D loss: 0.999975] [G loss: 1.000060]
epoch:0 step:2675[D loss: 0.999977] [G loss: 1.000062]
epoch:0 st

...

epoch:39 step:183880[D loss: 0.999895] [G loss: 1.000304]
epoch:39 step:183885[D loss: 0.999918] [G loss: 1.000315]
epoch:39 step:183890[D loss: 0.999946] [G loss: 1.000291]
epoch:39 step:183895[D loss: 0.999940] [G loss: 1.000145]
epoch:39 step:183900[D loss: 1.000029] [G loss: 1.000015]
epoch:39 step:183905[D loss: 1.000080] [G loss: 0.999905]
epoch:39 step:183910[D loss: 0.999993] [G loss: 0.999994]
epoch:39 step:183915[D loss: 1.000123] [G loss: 1.000037]
epoch:39 step:183920[D loss: 1.000142] [G loss: 0.999967]
epoch:39 step:183925[D loss: 0.999976] [G loss: 0.999960]
epoch:39 step:183930[D loss: 0.999929] [G loss: 1.000143]
epoch:39 step:183935[D loss: 0.999995] [G loss: 1.000231]
epoch:39 step:183940[D loss: 1.000018] [G loss: 1.000037]
epoch:39 step:183945[D loss: 0.999919] [G loss: 1.000177]
epoch:39 step:183950[D loss: 0.999965] [G loss: 1.000077]
epoch:39 step:183955[D loss: 0.999984] [G loss: 1.000129]
epoch:39 step:183960[D loss: 0.999914] [G loss: 1.000129]
epoch:39 step:

epoch:39 step:184590[D loss: 1.000000] [G loss: 1.000021]
epoch:39 step:184595[D loss: 0.999964] [G loss: 1.000048]
epoch:39 step:184600[D loss: 1.000048] [G loss: 0.999927]
epoch:39 step:184605[D loss: 1.000271] [G loss: 0.999779]
epoch:39 step:184610[D loss: 0.999960] [G loss: 1.000103]
epoch:39 step:184615[D loss: 0.999838] [G loss: 1.000162]
epoch:39 step:184620[D loss: 0.999908] [G loss: 1.000144]
epoch:39 step:184625[D loss: 0.999962] [G loss: 1.000117]
epoch:39 step:184630[D loss: 1.000057] [G loss: 0.999941]
epoch:39 step:184635[D loss: 0.999994] [G loss: 1.000016]
epoch:39 step:184640[D loss: 1.000198] [G loss: 0.999969]
epoch:39 step:184645[D loss: 0.999840] [G loss: 1.000186]
epoch:39 step:184650[D loss: 1.000005] [G loss: 1.000184]
epoch:39 step:184655[D loss: 1.000052] [G loss: 0.999974]
epoch:39 step:184660[D loss: 0.999991] [G loss: 1.000047]
epoch:39 step:184665[D loss: 0.999967] [G loss: 1.000057]
epoch:39 step:184670[D loss: 1.000035] [G loss: 1.000016]
epoch:39 step:

epoch:39 step:185300[D loss: 1.000182] [G loss: 0.999916]
epoch:39 step:185305[D loss: 0.999916] [G loss: 1.000083]
epoch:39 step:185310[D loss: 0.999945] [G loss: 1.000210]
epoch:39 step:185315[D loss: 1.000031] [G loss: 1.000129]
epoch:39 step:185320[D loss: 0.999894] [G loss: 1.000179]
epoch:39 step:185325[D loss: 0.999971] [G loss: 1.000093]
epoch:39 step:185330[D loss: 0.999935] [G loss: 1.000115]
epoch:39 step:185335[D loss: 0.999994] [G loss: 1.000089]
epoch:39 step:185340[D loss: 0.999946] [G loss: 1.000125]
epoch:39 step:185345[D loss: 0.999980] [G loss: 1.000056]
epoch:39 step:185350[D loss: 0.999962] [G loss: 1.000090]
epoch:39 step:185355[D loss: 0.999965] [G loss: 1.000093]
epoch:39 step:185360[D loss: 0.999970] [G loss: 1.000040]
epoch:39 step:185365[D loss: 1.000010] [G loss: 1.000054]
epoch:39 step:185370[D loss: 0.999943] [G loss: 1.000091]
epoch:39 step:185375[D loss: 0.999927] [G loss: 1.000073]
epoch:39 step:185380[D loss: 0.999978] [G loss: 1.000052]
epoch:39 step:

epoch:39 step:186010[D loss: 0.999963] [G loss: 1.000089]
epoch:39 step:186015[D loss: 1.000054] [G loss: 0.999897]
epoch:39 step:186020[D loss: 0.999926] [G loss: 1.000099]
epoch:39 step:186025[D loss: 0.999960] [G loss: 1.000090]
epoch:39 step:186030[D loss: 0.999992] [G loss: 1.000074]
epoch:39 step:186035[D loss: 0.999962] [G loss: 1.000072]
epoch:39 step:186040[D loss: 0.999985] [G loss: 1.000013]
epoch:39 step:186045[D loss: 0.999977] [G loss: 1.000069]
epoch:39 step:186050[D loss: 0.999972] [G loss: 1.000099]
epoch:39 step:186055[D loss: 0.999980] [G loss: 1.000039]
epoch:39 step:186060[D loss: 0.999966] [G loss: 1.000049]
epoch:39 step:186065[D loss: 0.999981] [G loss: 1.000099]
epoch:39 step:186070[D loss: 0.999963] [G loss: 1.000082]
epoch:39 step:186075[D loss: 0.999976] [G loss: 1.000052]
epoch:39 step:186080[D loss: 0.999997] [G loss: 1.000110]
epoch:39 step:186085[D loss: 0.999984] [G loss: 1.000053]
epoch:39 step:186090[D loss: 1.000013] [G loss: 1.000034]
epoch:39 step:

epoch:39 step:186720[D loss: 1.000121] [G loss: 0.999942]
epoch:39 step:186725[D loss: 1.000028] [G loss: 0.999908]
epoch:39 step:186730[D loss: 0.999963] [G loss: 1.000055]
epoch:39 step:186735[D loss: 1.000049] [G loss: 1.000054]
epoch:39 step:186740[D loss: 1.000073] [G loss: 1.000060]
epoch:39 step:186745[D loss: 0.999923] [G loss: 1.000147]
epoch:39 step:186750[D loss: 0.999984] [G loss: 1.000134]
epoch:39 step:186755[D loss: 1.000111] [G loss: 1.000099]
epoch:39 step:186760[D loss: 0.999885] [G loss: 1.000128]
epoch:39 step:186765[D loss: 0.999982] [G loss: 1.000072]
epoch:39 step:186770[D loss: 0.999963] [G loss: 1.000101]
epoch:39 step:186775[D loss: 0.999948] [G loss: 1.000090]
epoch:39 step:186780[D loss: 0.999958] [G loss: 1.000075]
epoch:39 step:186785[D loss: 0.999947] [G loss: 1.000096]
epoch:39 step:186790[D loss: 0.999976] [G loss: 1.000101]
epoch:39 step:186795[D loss: 1.000015] [G loss: 1.000011]
epoch:39 step:186800[D loss: 0.999947] [G loss: 1.000076]
epoch:39 step: